## Finding Hubs and Zones in Pricing Data

Discovering the Hubs or Zones for an ISO and then querying those location names are common first tasks when working with LMP data. This notebook shows some basic queries to help users of the gridstatusio API determine the correct `location_type` filters to use in order to get the correct locations for each ISO's hubs.

In [2]:
from gridstatusio import GridStatusClient

client = GridStatusClient()

## SPP

There are 2 hubs in SPP for pricing data.

*NOTE: The [`spp_lmp_real_time_by_bus`](https://www.gridstatus.io/datasets/spp_lmp_real_time_by_bus) dataset does not include Hubs.*

In [3]:
df_spp = client.get_dataset(
    dataset="spp_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Hub",
)
df_spp

2025-07-28 09:03:23 - INFO - Fetching Page 1...
2025-07-28 09:03:23 - INFO - GET https://api.gridstatus.io/v1/datasets/spp_lmp_real_time_5_min/query
2025-07-28 09:03:23 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:05:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'Hub', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:03:23 - INFO - Done in 0.22 seconds. 
2025-07-28 09:03:23 - INFO - Total number of rows: 2


interval_start_utc          interval_end_utc           market  \
0 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
1 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   

       location location_type       pnode      lmp   energy  congestion  \
0  SPPNORTH_HUB           Hub  SPPNORTH_H  12.4283  11.2583      1.2922   
1  SPPSOUTH_HUB           Hub  SPPSOUTH_H  -8.7672  11.2584    -19.2451   

     loss  
0 -0.1222  
1 -0.7805

## MISO
MISO labels many nodes as "Hub" in the raw data, but there are a much smaller subset that are actually state-level Hubs which people look for. To get down to those we filter on the `location_type` of "Hub" and then further filter the dataset based on location names.

In [5]:
df_miso = client.get_dataset(
    dataset="miso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Hub",
)

df_miso_hubs = df_miso[df_miso["location"].str.contains("HUB")]
df_miso_hubs

2025-07-28 09:03:40 - INFO - Fetching Page 1...
2025-07-28 09:03:40 - INFO - GET https://api.gridstatus.io/v1/datasets/miso_lmp_real_time_5_min/query
2025-07-28 09:03:40 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:05:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'Hub', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:03:40 - INFO - Done in 0.16 seconds. 
2025-07-28 09:03:40 - INFO - Total number of rows: 406


interval_start_utc          interval_end_utc           market  \
83  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
264 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
265 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
279 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
310 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
311 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
319 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
388 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   

          location location_type    lmp  energy  congestion  loss  
83    ARKANSAS.HUB           Hub  19.86   19.09        1.56 -0.79  
264   ILLINOIS.HUB           Hub   9.64   19.09       -9.11 -0.34  
265    INDIANA.HUB           Hub  27.19   19.09        7.19  0.91  
279  LOUISIANA.HUB           Hub  21.19   19.09        2.02  0.08  
310   MICHIGAN.HUB           Hub  26.60   19.09        6.10  1.41  
311       MINN.HUB           Hub  12.15   19.09       -6.45 -0.49  
319         MS.HUB           Hub  21.00   19.09        2.13 -0.22  
388      TEXAS.HUB           Hub  20.49   19.09        1.70 -0.30

## ERCOT
ERCOT has 5 Hubs and also publishes two averages, one each for Buses and Hubs.

In [6]:
df_ercot_hubs = client.get_dataset(
    dataset="ercot_spp_real_time_15_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:15",
    filter_column="location_type",
    filter_value="Trading Hub",
)
df_ercot_hubs

2025-07-28 09:04:30 - INFO - Fetching Page 1...
2025-07-28 09:04:30 - INFO - GET https://api.gridstatus.io/v1/datasets/ercot_spp_real_time_15_min/query
2025-07-28 09:04:30 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:15:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'Trading Hub', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:04:31 - INFO - Done in 0.17 seconds. 
2025-07-28 09:04:31 - INFO - Total number of rows: 7


interval_start_utc          interval_end_utc    location  \
0 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00   HB_BUSAVG   
1 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00  HB_HOUSTON   
2 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00   HB_HUBAVG   
3 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00    HB_NORTH   
4 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00      HB_PAN   
5 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00    HB_SOUTH   
6 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00     HB_WEST   

  location_type            market    spp  
0   Trading Hub  REAL_TIME_15_MIN  19.96  
1   Trading Hub  REAL_TIME_15_MIN  20.30  
2   Trading Hub  REAL_TIME_15_MIN  20.13  
3   Trading Hub  REAL_TIME_15_MIN  19.86  
4   Trading Hub  REAL_TIME_15_MIN  11.57  
5   Trading Hub  REAL_TIME_15_MIN  19.39  
6   Trading Hub  REAL_TIME_15_MIN  20.98

ERCOT also publishes data for several load zones

In [7]:
df_ercot_zones = client.get_dataset(
    dataset="ercot_spp_real_time_15_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:15",
    filter_column="location_type",
    filter_value="Load Zone",
)
df_ercot_zones

2025-07-28 09:04:52 - INFO - Fetching Page 1...
2025-07-28 09:04:52 - INFO - GET https://api.gridstatus.io/v1/datasets/ercot_spp_real_time_15_min/query
2025-07-28 09:04:52 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:15:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'Load Zone', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:04:52 - INFO - Done in 0.14 seconds. 
2025-07-28 09:04:52 - INFO - Total number of rows: 8


interval_start_utc          interval_end_utc    location  \
0 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00      LZ_AEN   
1 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00      LZ_CPS   
2 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00  LZ_HOUSTON   
3 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00     LZ_LCRA   
4 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00    LZ_NORTH   
5 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00    LZ_RAYBN   
6 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00    LZ_SOUTH   
7 2024-04-01 00:00:00+00:00 2024-04-01 00:15:00+00:00     LZ_WEST   

  location_type            market    spp  
0     Load Zone  REAL_TIME_15_MIN  20.86  
1     Load Zone  REAL_TIME_15_MIN  21.31  
2     Load Zone  REAL_TIME_15_MIN  20.31  
3     Load Zone  REAL_TIME_15_MIN  20.49  
4     Load Zone  REAL_TIME_15_MIN  19.83  
5     Load Zone  REAL_TIME_15_MIN  19.72  
6     Load Zone  REAL_TIME_15_MIN  23.03  
7     Load Zone  REAL_TIME_15_MIN  22.93

## CAISO
CAISO has 3 Trading Hubs in their LMP data.

In [8]:
df_caiso = client.get_dataset(
    dataset="caiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Trading Hub",
)
df_caiso

2025-07-28 09:04:59 - INFO - Fetching Page 1...
2025-07-28 09:04:59 - INFO - GET https://api.gridstatus.io/v1/datasets/caiso_lmp_real_time_5_min/query
2025-07-28 09:04:59 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:05:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'Trading Hub', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:05:00 - INFO - Done in 1.59 seconds. 
2025-07-28 09:05:00 - INFO - Total number of rows: 3


interval_start_utc          interval_end_utc           market  \
0 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
1 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
2 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   

           location location_type      lmp   energy  congestion     loss   ghg  
0  TH_NP15_GEN-APND   Trading Hub -3.99571 -3.29997    -0.70498  0.00924  None  
1  TH_SP15_GEN-APND   Trading Hub -3.16203 -3.29997     0.00000  0.13794  None  
2  TH_ZP26_GEN-APND   Trading Hub -3.14685 -3.29997     0.00000  0.15312  None

CAISO also publishes data for several DLAPs (Default Load Aggregation Points) which is the node at which all bids for demand are submitted and settled

In [9]:
df_caiso_dlap = client.get_dataset(
    dataset="caiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="DLAP",
)
df_caiso_dlap

2025-07-28 09:05:10 - INFO - Fetching Page 1...
2025-07-28 09:05:10 - INFO - GET https://api.gridstatus.io/v1/datasets/caiso_lmp_real_time_5_min/query
2025-07-28 09:05:10 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:05:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'DLAP', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:05:12 - INFO - Done in 2.42 seconds. 
2025-07-28 09:05:12 - INFO - Total number of rows: 6


interval_start_utc          interval_end_utc           market  \
0 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
1 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
2 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
3 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
4 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
5 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   

             location location_type      lmp   energy  congestion     loss  \
0  DLAP_PACE_NPM-APND          DLAP -3.29844 -3.29997     0.00120  0.00033   
1  DLAP_PACW_NPM-APND          DLAP -3.29745 -3.29997     0.00120  0.00132   
2      DLAP_PGAE-APND          DLAP -4.27954 -3.29997    -0.93997 -0.03960   
3       DLAP_SCE-APND          DLAP -3.23694 -3.29997     0.00000  0.06303   
4      DLAP_SDGE-APND          DLAP -3.24420 -3.29997     0.00000  0.05577   
5       DLAP_VEA-APND          DLAP -3.09768 -3.29997     0.00000  0.20229   

    ghg  
0  None  
1  None  
2  None  
3  None  
4  None  
5  None

## PJM
As the largest market in the US by load, PJM also has a higher number of hubs than most markets.

In [10]:
df_pjm = client.get_dataset(
    dataset="pjm_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="HUB",
)
df_pjm

2025-07-28 09:05:37 - INFO - Fetching Page 1...
2025-07-28 09:05:37 - INFO - GET https://api.gridstatus.io/v1/datasets/pjm_lmp_real_time_5_min/query
2025-07-28 09:05:37 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:05:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'HUB', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:05:37 - INFO - Done in 0.24 seconds. 
2025-07-28 09:05:37 - INFO - Total number of rows: 12


interval_start_utc          interval_end_utc           market  \
0  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
1  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
2  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
3  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
4  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
5  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
6  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
7  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
8  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
9  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
10 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
11 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   

           location  location_id location_short_name location_type    lmp  \
0    AEP-DAYTON HUB     34497127      AEP-DAYTON HUB           HUB  31.68   
1       AEP GEN HUB     34497125         AEP GEN HUB           HUB  31.45   
2      ATSI GEN HUB    116013751        ATSI GEN HUB           HUB  31.46   
3   CHICAGO GEN HUB     33092311     CHICAGO GEN HUB           HUB  24.69   
4       CHICAGO HUB     33092313         CHICAGO HUB           HUB  25.32   
5      DOMINION HUB     35010337        DOMINION HUB           HUB  31.95   
6       EASTERN HUB        51217         EASTERN HUB           HUB  32.14   
7    NEW JERSEY HUB      4669664      NEW JERSEY HUB           HUB  31.89   
8    N ILLINOIS HUB     33092315      N ILLINOIS HUB           HUB  25.11   
9          OHIO HUB     34497151            OHIO HUB           HUB  31.63   
10      WESTERN HUB        51288         WESTERN HUB           HUB  32.12   
11     WEST INT HUB        51287        WEST INT HUB           HUB  31.67   

    energy  congestion  loss  
0    31.26        0.41  0.01  
1    31.26        0.75 -0.56  
2    31.26        0.46 -0.26  
3    31.26       -5.18 -1.39  
4    31.26       -4.94 -1.00  
5    31.26        0.74 -0.05  
6    31.26        0.62  0.26  
7    31.26        0.60  0.03  
8    31.26       -5.01 -1.14  
9    31.26        0.30  0.07  
10   31.26        0.64  0.22  
11   31.26        0.63 -0.22

## NYISO
NYISO doesn't calculate and publish hubs as a separate category. The market also doesn't price at every bus (only at generators and at the zonal level), so we will get the Zone names instead. This list also includes external regions such as Ontario (OH) and Quebec (HQ).

In [12]:
df_nyiso = client.get_dataset(
    dataset="nyiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Zone",
)
df_nyiso

2025-07-28 09:06:30 - INFO - Fetching Page 1...
2025-07-28 09:06:30 - INFO - GET https://api.gridstatus.io/v1/datasets/nyiso_lmp_real_time_5_min/query
2025-07-28 09:06:30 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:05:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'Zone', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:06:31 - INFO - Done in 0.16 seconds. 
2025-07-28 09:06:31 - INFO - Total number of rows: 15


interval_start_utc          interval_end_utc           market  \
0  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
1  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
2  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
3  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
4  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
5  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
6  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
7  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
8  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
9  2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
10 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
11 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
12 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
13 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   
14 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  REAL_TIME_5_MIN   

   location location_type    lmp  energy  congestion  loss  
0    CAPITL          Zone  20.13   19.58       -0.00  0.55  
1    CENTRL          Zone  19.60   19.58       -0.00  0.02  
2    DUNWOD          Zone  20.56   19.58       -0.00  0.98  
3    GENESE          Zone  19.31   19.58       -0.00 -0.27  
4       H Q          Zone  19.58   19.58       -0.00  0.00  
5    HUD VL          Zone  20.48   19.58       -0.00  0.90  
6    LONGIL          Zone  20.66   19.58       -0.00  1.08  
7    MHK VL          Zone  20.15   19.58       -0.00  0.57  
8    MILLWD          Zone  20.50   19.58       -0.00  0.92  
9     NORTH          Zone  19.62   19.58       -0.00  0.04  
10      NPX          Zone  19.92   19.58       -0.03  0.37  
11   N.Y.C.          Zone  20.68   19.58       -0.00  1.10  
12      O H          Zone  18.98   19.59       -0.00 -0.61  
13      PJM          Zone  19.86   19.59       -0.00  0.27  
14     WEST          Zone  19.39   19.59       -0.00 -0.20

## ISO-NE
ISO-NE only has a single internal Hub.

In [13]:
df_isone = client.get_dataset(
    dataset="isone_lmp_real_time_5_min_prelim",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="HUB",
)
df_isone

2025-07-28 09:07:06 - INFO - Fetching Page 1...
2025-07-28 09:07:06 - INFO - GET https://api.gridstatus.io/v1/datasets/isone_lmp_real_time_5_min_prelim/query
2025-07-28 09:07:06 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:05:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'HUB', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:07:07 - INFO - Done in 0.18 seconds. 
2025-07-28 09:07:07 - INFO - Total number of rows: 1


interval_start_utc          interval_end_utc         location  \
0 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  .H.INTERNAL_HUB   

  location_type    lmp  energy  congestion  loss  
0           HUB  34.91   34.69        0.05  0.17

In addition, ISO-NE has 8 Load Zones.

In [15]:
df_isone_load_zone = client.get_dataset(
    dataset="isone_lmp_real_time_5_min_prelim",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="LOAD ZONE",
)
df_isone_load_zone

2025-07-28 09:08:47 - INFO - Fetching Page 1...
2025-07-28 09:08:47 - INFO - GET https://api.gridstatus.io/v1/datasets/isone_lmp_real_time_5_min_prelim/query
2025-07-28 09:08:47 - INFO - Params: {'start_time': Timestamp('2024-04-01 00:00:00'), 'end_time': Timestamp('2024-04-01 00:05:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'LOAD ZONE', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-07-28 09:08:47 - INFO - Done in 0.11 seconds. 
2025-07-28 09:08:47 - INFO - Total number of rows: 8


interval_start_utc          interval_end_utc         location  \
0 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00   .Z.CONNECTICUT   
1 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00         .Z.MAINE   
2 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00    .Z.NEMASSBOST   
3 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00  .Z.NEWHAMPSHIRE   
4 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00   .Z.RHODEISLAND   
5 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00        .Z.SEMASS   
6 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00       .Z.VERMONT   
7 2024-04-01 00:00:00+00:00 2024-04-01 00:05:00+00:00        .Z.WCMASS   

  location_type    lmp  energy  congestion  loss  
0     LOAD ZONE  34.01   34.69        0.05 -0.73  
1     LOAD ZONE  34.01   34.69       -0.39 -0.29  
2     LOAD ZONE  35.27   34.69        0.05  0.53  
3     LOAD ZONE  34.98   34.69        0.05  0.24  
4     LOAD ZONE  34.81   34.69        0.05  0.07  
5     LOAD ZONE  35.20   34.69        0.05  0.46  
6     LOAD ZONE  34.63   34.69        0.05 -0.11  
7     LOAD ZONE  34.84   34.69        0.05  0.10

## Now let's print out all of the locations rom each market we just generated...

In [23]:
def print_locations(title: str, locations: list[str]) -> None:
    print(f"\n{title}:")
    for loc in locations:
        print(f"  {loc}")


print_locations("CAISO Trading Hub", list(df_caiso["location"].unique()))
print_locations("CAISO DLAP", list(df_caiso_dlap["location"].unique()))
print_locations("ERCOT Trading Hub", list(df_ercot_hubs["location"].unique()))
print_locations("ERCOT Load Zone", list(df_ercot_zones["location"].unique()))
print_locations("SPP", list(df_spp["location"].unique()))
print_locations("MISO", list(df_miso_hubs["location"].unique()))
print_locations("PJM", list(df_pjm["location"].unique()))
print_locations("NYISO", list(df_nyiso["location"].unique()))
print_locations("ISONE Hub", list(df_isone["location"].unique()))
print_locations("ISONE Load Zone", list(df_isone_load_zone["location"].unique()))


CAISO Trading Hub:
  TH_NP15_GEN-APND
  TH_SP15_GEN-APND
  TH_ZP26_GEN-APND

CAISO DLAP:
  DLAP_PACE_NPM-APND
  DLAP_PACW_NPM-APND
  DLAP_PGAE-APND
  DLAP_SCE-APND
  DLAP_SDGE-APND
  DLAP_VEA-APND

ERCOT Trading Hub:
  HB_BUSAVG
  HB_HOUSTON
  HB_HUBAVG
  HB_NORTH
  HB_PAN
  HB_SOUTH
  HB_WEST

ERCOT Load Zone:
  LZ_AEN
  LZ_CPS
  LZ_HOUSTON
  LZ_LCRA
  LZ_NORTH
  LZ_RAYBN
  LZ_SOUTH
  LZ_WEST

SPP:
  SPPNORTH_HUB
  SPPSOUTH_HUB

MISO:
  ARKANSAS.HUB
  ILLINOIS.HUB
  INDIANA.HUB
  LOUISIANA.HUB
  MICHIGAN.HUB
  MINN.HUB
  MS.HUB
  TEXAS.HUB

PJM:
  AEP-DAYTON HUB
  AEP GEN HUB
  ATSI GEN HUB
  CHICAGO GEN HUB
  CHICAGO HUB
  DOMINION HUB
  EASTERN HUB
  NEW JERSEY HUB
  N ILLINOIS HUB
  OHIO HUB
  WESTERN HUB
  WEST INT HUB

NYISO:
  CAPITL
  CENTRL
  DUNWOD
  GENESE
  H Q
  HUD VL
  LONGIL
  MHK VL
  MILLWD
  NORTH
  NPX
  N.Y.C.
  O H
  PJM
  WEST

ISONE Hub:
  .H.INTERNAL_HUB

ISONE Load Zone:
  .Z.CONNECTICUT
  .Z.MAINE
  .Z.NEMASSBOST
  .Z.NEWHAMPSHIRE
  .Z.RHODEISLAND
  .Z.SEMAS